In [1]:
import os
import numpy as np
from sys import getsizeof

In [2]:
!ls checkpoints/ -la

total 20
drwxr-xr-x 2 kuzmenko dbvis 4096 Feb 26 13:40 .
drwxr-xr-x 9 kuzmenko dbvis 4096 Feb 26 14:25 ..
-rw-r--r-- 1 kuzmenko dbvis 6148 Dec  2  2019 .DS_Store
-rw-r--r-- 1 kuzmenko dbvis   70 Dec  2  2019 ._.DS_Store


In [12]:
!ls

LICENSE		      __init__.py  data		   loader.py	   stock_set
README.md	      __pycache__  input	   ops.py
Save_np_arrays.ipynb  checkpoints  load_npy.ipynb  stock_model.py


In [33]:
!rm -rf checkpoints/*

In [3]:
import re
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

In [13]:
path = "data"
stocks_set = np.zeros([0,2,128,5])
labels_set = np.zeros([0,2])
for dir_item in sorted(os.listdir(path), key=numericalSort):
    dir_item_path = os.path.join(path, dir_item)
    if os.path.isfile(dir_item_path):
        print ("Current File Being Processed is: " + dir_item)
        if("stock" in dir_item):
            ss = np.load(dir_item_path)
            stocks_set = np.concatenate((stocks_set, ss), axis=0)
        else:
            ls = np.load(dir_item_path)
            labels_set = np.concatenate((labels_set, ls), axis=0)

Current File Being Processed is: label_set_debug_1.0.npy
Current File Being Processed is: stock_set_debug_1.0.npy


In [14]:
print(stocks_set.nbytes/1048576,stocks_set.shape,labels_set.shape)


28.046875 (2872, 2, 128, 5) (2872, 2)


In [15]:
up = 0
down = 0
idxes_to_take =[]
for idx,val in enumerate(labels_set):
    if val[0]==1.:
        up +=1
#         idxes_to_take.append(idx)
    else: 
        down +=1
#         if(up>down):
#             idxes_to_take.append(idx)
# print(" idxes_to_take ", idxes_to_take)
print("labels_set len: %d, Prise risen: %d, prise fall: %d" %(len(labels_set), up, down))

labels_set len: 2872, Prise risen: 1910, prise fall: 962


In [16]:
# shuffling the data
perm = np.arange(labels_set.shape[0])
np.random.shuffle(perm)
stocks_set = stocks_set[perm]
labels_set = labels_set[perm]


# normalize the data
stocks_set_ = np.zeros(stocks_set.shape)
for i in range(len(stocks_set)):
    min = stocks_set[i].min(axis=(0,1), keepdims=True)
    max = stocks_set[i].max(axis=(0,1), keepdims=True)
    stocks_set_[i] = (stocks_set[i] - min) / (max - min)
stocks_set = stocks_set_

train_test_ratio=4.0
# selecting 1/5 for testing, and 4/5 for training
train_test_idx = int((1.0 / (train_test_ratio + 1.0)) * labels_set.shape[0])
train_stocks = stocks_set[train_test_idx:,:,:]
train_labels = labels_set[train_test_idx:]
test_stocks = stocks_set[:train_test_idx,:,:]
test_labels = labels_set[:train_test_idx]

In [17]:
np.save('input/train_stocks_debug',train_stocks)
np.save('input/train_labels_debug',train_labels)
np.save('input/test_stocks_debug',test_stocks)
np.save('input/test_labels_debug',test_labels)

In [18]:
!ls 

LICENSE		      __init__.py  data		   loader.py	   stock_set
README.md	      __pycache__  input	   ops.py
Save_np_arrays.ipynb  checkpoints  load_npy.ipynb  stock_model.py


In [ ]:
! python3 stock_model.py

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
First layer output:  Tensor("prediction/model/conv_1/Relu:0", shape=(?, 2, 128, 128), dtype=float32)
2018-02-26 17:41:14.828672: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2018-02-26 17:41:15.306387: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1030] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1.3285
pciBusID: 0000:07:00.0
totalMemory: 15.89GiB freeMemory: 14.89GiB
2018-02-26 17:41:15.638205: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1030] Found device 1 with properties: 
name: Quadro M6000 major: 5